# Storage: caching; loop optimization
_COSC 208, Introduction to Computer Systems, Fall 2025_

## Caching

* _Why do we have caches on the CPU?_ — accessing main memory is ~100x slower than accessing a register
* Store instructions and data (stack, heap, etc.) from main memory
* Three levels — L1, L2, and L3
* Range in size from a few KB to a few MB – L1 is smallest and fastest; L3 is largest and slowest
* Cache line (i.e., cache entry) is typically larger than a word — e.g., 128 bytes
    * _Why?_ — spatial locality
* What happens when we write to memory?
    * Write through cache — write to the cache and main memory
    * Write back cache — initially write to the cache; write to main memory when the entry is evicted from the cache
    * What are the advantages of each approach?
        * Write through cache ensures consistency between CPU cores
        * Write back cache only incurs the overhead of accessing main memory when absolutely necessary
* Caches are also used in many other places in computer systems – e.g., web browser cache, content distribution network (CDN)

Q1: _For each of the following instances of caching, indicate whether the caching is motivated by temporal or spatial locality._

* A CPU caches the first 32 instructions of a function when the function is called – spatial
* A CPU caches all of the instructions for a frequently called function – temporal
* A web browser caches the Moodle pages for your courses, which you view multiple times per week – temporal
* A content distribution network (CDN) caches a video that has gone viral – temporal
* A content distribution network (CDN) caches "recommended videos" related to a video – spatial

* A CPU caches the first 32 instructions of a function when the function is called
* A CPU caches all of the instructions for a frequently called function
* A web browser caches the Moodle pages for your courses, which you view multiple times per week
* A content distribution network (CDN) caches a video that has gone viral
* A content distribution network (CDN) caches "recommended videos" related to a video

<p style="height:5em;"></p>

## Cache replacement

* If a cache is full, then a cache entry must be removed so different data can be placed in the cache
* Cache replacement policy governs which data is removed
* _What should a good cache replacement policy do?_ — maximize the number of cache hits (or minimize the number of cache misses)
    * Evaluation metric: Hit ratio = number of hits / total number of memory accesses
* _How do we determine which cache entry to replace?_
* First-in First-out (FIFO)
    * Simple to implement
    * Doesn’t consider the importance of a cache entry
* Optimal replacement policy – replace the entry that will be accessed furthest in the future
    * Impractical because we don’t know data access patterns a priori
* Least Recently Used (LRU)
    * LRU assumes an item that was accessed recently will be accessed again soon – temporal locality
    * Downside: lots of overhead to implement — need to store an ordered list of items and move an item up in the list whenever it’s accessed
    * Where does this go wrong? — when working-set size (i.e., number of repeatedly accessed entries) is (slightly) greater than size of the cache

Assume a cache can hold 3 entries and the following 15 data accesses occur: 
```
3, 4, 4, 5, 3, 2, 3, 4, 1, 4, 4, 2, 5, 2, 4
```
Q2: _What is the sequence of hits, insertions, and replacements that occur when an **optimal** cache replacement algorithm is used?_

```
+3, +4, H4, +5, H3, -5/+2, H3, H4, -3/+1, H4, H4, H2, -1/+5, H2, H4
Hit ratio = 9/15 = 60%
```

<p style="height:8em;"></p>

Q3: _What is the sequence of hits, insertions, and replacements that occur when a **least recently used (LRU)** cache replacement algorithm is used?_

```
+3, +4, H4, +5, H3, -4/+2, H3, -5/+4, -2/+1, H4, H4, -3/+2, -1/+5, H2, H4
Hit ratio = 7/15 = 47%
```

<p style="height:8em;"></p>

Q4: _What is the sequence of hits, insertions, and replacements that occur when a **first in first out (FIFO)** cache replacement algorithm is used?_

```
+3, +4, H4, +5, H3, -3/+2, -4/+3, -5/+4, -2/+1, H4, H4, -3/+2, -4/+5, H2, -1/+4
Hit ratio = 5/15 = 33%
```

<p style="height:8em;"></p>

Assume a cache can hold 3 entries and the following 12 data accesses occur: 
```
2, 4, 1, 2, 4, 3, 2, 4, 1, 2, 4, 1
```
Q5: _What is the sequence of hits, insertions, and replacements that occur when an **optimal** cache replacement algorithm is used?_

```
+2, +4, +1, H2, H4, -1/+3, H2, H4, -3/+1, H2, H4, H1
```

<p style="height:8em;"></p>

Q6: _What is the sequence of hits, insertions, and replacements that occur when a **least recently used (LRU)** cache replacement algorithm is used?_

```
+2, +4, +1, H2, H4, -1/+3, H2, H4, -3/+1, H2, H4, H1
```

<p style="height:8em;"></p>

## Loop optimization

_Example_

In [1]:
#include <stdlib.h>
#include <stdio.h>
#define LEN 12
int main() {
    int *array = malloc(sizeof(int) * LEN);

    for (int i = 0; i < LEN; i++) {
        array[i] = i;
    }
    
    int sum = 0;
    for (int j = 0; j < 4; j++) {
        for (int k = 0; k < LEN; k += 4) {
            sum += array[j+k];
        }
    }
    printf("%d\n", sum);
}

66


* _Assume the values of all local variables are stored in registers (**not** the stack) and the value of `array` is `0x400`. What is the sequence of memory addresses that are accessed?_
    * First for loop: `0x400`, `0x404`, `0x408`, `0x40c`, `0x410`, `0x414`, `0x418`, `0x41c`, `0x420`, `0x424`, `0x428`, `0x42c`
    * Second for loop: `0x400`, `0x410`, `0x420`, `0x404`, `0x414`, `0x424`, `0x408`, `0x418`, `0x428`, `0x40c`, `0x41c`, `0x42c`, 
    * Notice that the first for loop accesses memory addresses in order, whereas the second for loop accesses addresses out of order
* _Now assume the system uses a cache that holds 2 entries which are each 16 bytes large. What is the sequence of hits and misses using a least recently used (LRU) replacement policy?_
    * First for loop: Miss (+0x4000), Hit, Hit, Hit, Miss (+0x4010), Hit, Hit, Hit, Miss (-0x4000/+0x4020), Hit, Hit, Hit
    * Second for loop: Miss (-0x4010/+0x4000), Miss (-0x4020/+0x4010), Miss (-0x4000/+0x4020), Miss (-0x4010/+0x4000), Miss (-0x4020/+0x4010), Miss (-0x4000/+0x4020), Miss (-0x4010/+0x4000), Miss (-0x4020/+0x4010), Miss (-0x4000/+0x4020), Miss (-0x4010/+0x4000), Miss (-0x4020/+0x4010), Miss (-0x4000/+0x4020)
    * Notice that the first for loop has three hits after each miss, whereas the second for loop is all misses
* _How could we modify the code to achieve a higher hit ratio?_ – loop interchange, i.e., swap inner and outer loops

In [2]:
#include <stdlib.h>
#include <stdio.h>
#define LEN 12
int main() {
    int *array = malloc(sizeof(int) * LEN);

    for (int i = 0; i < LEN; i++) {
        array[i] = i;
    }
    
    int sum = 0;
    for (int k = 0; k < LEN; k += 4) {
        for (int j = 0; j < 4; j++) {
            sum += array[j+k];
        }
    }
    printf("%d\n", sum);
}

66


* Loop fusion — combine two loops at the same level into a single loop

<p style="height:9em;"></p>

Q7: _Should loop interchange, loop fusion, neither, or both be applied to this code?_

In [ ]:
void hundreds() {
    int *nums = malloc(sizeof(int) * 1000);
    for (int i = 0; i < 1000; i+= 100) {
        for (int j = 0; j < 100; j++) {
            nums[i+j] = i;
        }
    }
}

Neither – loop interchange would reduce the ability to leverage spatial locality; there are not loops to fuse

<p style="height:2em;"></p>

Q8: _Should loop interchange, loop fusion, neither, or both be applied to this code?_

In [ ]:
void multiplication(int grid[][], int rows, int cols) {
    for (int c = 0; c < cols; c++) {
        for (int r = 0; r < rows; r++) {
            grid[r][c] = c * r;
        }
    }
}

Loop interchange will better leverage spatial locality; there are no loops to fuse

<p style="height:2em;"></p>

Q9: _Should loop interchange, loop fusion, neither, or both be applied to this code?_

In [ ]:
int odds(int *nums, int length) {
    for (int i = 0; i < length; i++) {
        nums[i] = nums[i] % 2;
    }
    int count = 0;
    for (int j = 0; j < length; j++) {
        count += nums[j];
    }
    return count;
}

Loop fusion will better leverage temporal locality; there are no loops to interchange

<p style="height:20em;"></p>

Q10: _Should loop interchange, loop fusion, neither, or both be applied to this code?_

In [ ]:
long stdev(int *nums, int length) {
    long sum = 0;
    for (int i = 0; i < length; i++) {
        sum += nums[i];
    }
    int mean = sum / length;
    sum = 0;
    for (int j = 0; j < length; j++) {
        int diff = nums[j] - mean;
        sum += diff * diff:
    }
    mean = sum / length;
    return sqrt(mean);
}

Neither — there are no loops to interchange; the loops cannot be fused, because the second loop depends on the result of the first loop